# Sprint16課題 論文読解入門


以下の論文を読み問題に答えてください。CNNを使った物体検出（Object Detection）の代表的な研究です。

[8]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99

https://arxiv.org/pdf/1506.01497.pdf

(1) 物体検出の分野にはどういった手法が存在したか。<br>

* R-CNN
* SPPnet
* Fast R-CNN

<br>
など

(2) Fasterとあるが、どういった仕組みで高速化したのか。

2つの学習を一部共通化することで学習にかかるコストを下げた。<br>
具体的には物体の存在する候補領域を決定するための学習(region proposal)と、それによって提案された領域に存在する物体を検知する学習（Object detection）の2つの学習において、従来は別々の学習を行なっていたところを2つの学習で共通したConvolution層を用いることでコストを下げた（detectionが律速なので、共通化することでproposalのコストがほぼゼロになった）。

(3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

本論文の手法に代表されるTwo-stageの手法では、クラスに関係ない物体候補領域の提案のステージと、クラス分けを伴うDetectionのステージに別れている。OverFeatに代表されるOne-stageの手法では前者のregion proposalのステージがなく、Sliding windonなどから与えられた単なる窓から、直接Detectionでクラス分けをする。Two-stageの方が精度が高い（論文には書いていないが、あらかじめ物体が存在するとわかっている窓に対してClassificationをするため?）。また、One-stageの方がTwo-stageの方が速度も速い（同じ精度を出すために処理しなければいけないregion proposalの数が少ないから?）。

(4) RPNとは何か。<br>
この論文で筆者らが導入したRegion proposal (object proposalとも)の一種。この手法によって、classificationのために画像からCNNで抽出した特徴量を、region proposalのためにも利用できるようになった。


* image-pyramids (DPM, HOGなど)： 
    画像のサイズを様々に変えることで切り取る領域を変える。一般に時間的コストが高い。
* sliding-windows (Fast R-CNNなど): 
    画像に当てるFilterのサイズを様々に変えることで取る領域を変える（filter-pyramids）。
* Region proposal network(Faster R-CNNなど):
    画像をCNN上の特徴マップに落としこんでから切り取る(anchor-pyramids)


(5) RoIプーリングとは何か。<br>
RPNで得られたbounding boxに関する特徴量情報（RoI: Region of Interest）はサイズがバラバラなので（犬と車ではサイズが違うみたいな事?）、classificationと共有している特徴量マップに合流させる際にサイズを一定サイズに揃える必要がある。RoIプーリング層を通す事でそのような変換が可能と成る。


(6) Anchorのサイズはどうするのが適切か。<br>
検出したいものによって変える必要はない（Defaultの3 scale x 3 ratioでだいたいイケる）。特徴量マップに対して当てるから？

(7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

PASCAL VOCやMS COCOのデータで先行研究との比較を行なっている。mAP ([mean average precision](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173))が比較の際の指標値として用いられており、Proposalの数を減らしても従来法よりも良い結果が得られている。<br>


画像認識のベンチマーク測定によく使われる画像セット<br>
* PASCAL VOC
* Microsoft COCO: こっちの方が複雑

(8) （アドバンス）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。<br>
YOLOなどの高速で物体認識をすることを目的とした手法の論文で、ベンチマークの比較対象となる先行研究として引用されている

メモ


#### R-CNN -> Fast R-CNN -> Faster R-CNNの流れ
物体検出はLocalizationとClassificationから成る。CNNでは、画像の分類をするClassificationはできても、画像上で物体が占める位置を特定するLocalizationは難しいと考えられてきた。R-CNN（Region-based CNN）ではLocalizationにCNNが応用された。sliding windowで生成された多数の窓をselective　searchで絞り込み、CNNにinputするbounding boxの数を絞り込んだ。また、bounding boxを決まった大きさの正方形にリサイズして統一してからinputすることでCNNを位置決定に応用できるようになった。
R-CNNによってCNNを物体検知に応用できるようになり大きく精度向上したが、R-CNNでは学習を位置特定とクラス分類の2つ別々に実行する必要があり、また、実行時間も遅かった。その原因は、R-CNNではSelective searchでもまだ２０００枚のbounding boxを処理する必要があったためであるが、SPP-Netではこの点が改善された。具体的にはwindowとして切り取っていない画像全体をCNNに通し、得られた特徴量マップから物体領域の情報を抽出するSPP (Spatial Pyramid Pooling)を取り入れた。SPP-NetによりR-CNNより数十倍速い処理が可能となったが、位置特定のためのregressionと物体判別のためのclassificationの2つの計算を別々に実行する点は変わらなかった。Fast R-CNNではこの点が解消され全ての層が1つなぎになり、end-to-endでの学習が可能になった。これは、RoI pooling層の導入によりLocalizationに必要な位置の情報を、Classificationで用いるCNNの特徴マップ上に含めることに成功し、また、R-CNNのClassificationを担っていたSVMをCNNの流れの後の全結合層内のSoftmaxに置換することに成功したためである。
さらに、1つのlossで2つの計算をまとめて評価するmulti-task lossを導入することで、学習全体の重み更新を一度に行えるようになった。
Faster R-CNNでは、ClassificationとLocalizationのCNNの共通利用を可能とし、さらに高速となった。Fast R-CNNでは、Localizationのために画像からregion proposalを生成する処理と、Classificationのために画像から特徴マップを作る処理が別々に行われていた。Faster R-CNNでは、Classificationのために画像から生成された特徴マップを用いて、region proposalを抽出するRPN （region proposal network）の手法を導入することで、ClassificationとLocalizationの2つの処理に独立して存在していたCNNの処理の共通化に成功した。

#### End-to-end
end-to-endとは、入力と出力のみで、途中の過程にある全てのステップを含めて学習してしまうこと。
文字認識は、①画像からテキストであることを検出し、②テキスト内の文字を分割、③分割した文字を認識させるなどの複数ステップのタスクからなるが、これらのステップを省いてまとめて学習させるような手法はend-to-endであるといえる。
一般に、end-to-endのタスクは、そうで無いタスクよりも多くのデータセットが必要となることが多い。多くのデータが用意できないときは従来法を採る。(例：画像→テキスト検出→文字分割→文字認識)


#### region proposal
sliding windowで様々な位置、スケール、アスペクト比の多数の窓をスライドして網羅的に調べるコストがかかるが、物体が存在しそうな領域だけを提案できれば効率的に物体検出などができる。できるだけ少ない数の提案で全ての物体をカバーできるようなregion proposalの確立が目指されてきた。
従来法のimage-pyramidsとfilter-pyramids (sliding-window)と違い、本論文ではanchor pyramidsでclassificationと層を共有する事でコスト効率の良い方法を確立した。


#### 略語等
* VGG: Visual geometry group
* EB: Edge box
* Region proposal: 物体候補領域提案手法
* Selective　search: sliding windowでサイズ、位置、アスペクト比などを変えて網羅的に探索するのでなく、ピクセルレベルでの類似領域をまとめることでregion proposalの数を大幅に減らす手法
* ground truth: 正解ラベル。画像上でのBinding boxの正しい位置。
* RoI: Region of interest
* cross-boundary: Anchorが枠から出てしまう場合
* IoU：Intersection over union. Measures the overlap area between two boundaries（predict / ground truth）
* NMS ([Non-max supression](http://pynote.hatenablog.com/entry/opencv-non-maximum-suppression)): 同じ物体に対して複数の似たようなBounding boxができてしまうことに対して、もっともIoUが高いBoxのみを残して、それと似ているBoxを削除する操作のこと。





参考<br>
[深層学習をめぐる最近の熱狂](https://www.jstage.jst.go.jp/article/psychono/35/2/35_35.26/_pdf)<br>